In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

file_path = 'sherlock-holm.es_stories_plain-text_advs.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
total_words

8200

In [4]:
input_sequences = [tokenizer.texts_to_sequences([line])[0][:i+1] for line in text.split('\n') for i in range(1, len(tokenizer.texts_to_sequences([line])[0]))]

In [5]:
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre', truncating='post', dtype='int32')

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 170s 54ms/step - loss: 6.2070 - accuracy: 0.0809
Epoch 2/100
3010/3010 [==============================] - 165s 55ms/step - loss: 5.4804 - accuracy: 0.1254
Epoch 3/100
3010/3010 [==============================] - 161s 53ms/step - loss: 5.1041 - accuracy: 0.1469
Epoch 4/100
3010/3010 [==============================] - 184s 61ms/step - loss: 4.7745 - accuracy: 0.1665
Epoch 5/100
3010/3010 [==============================] - 180s 60ms/step - loss: 4.4721 - accuracy: 0.1846
Epoch 6/100
3010/3010 [==============================] - 211s 70ms/step - loss: 4.1877 - accuracy: 0.2052
Epoch 7/100
3010/3010 [==============================] - 203s 67ms/step - loss: 3.9155 - accuracy: 0.2293
Epoch 8/100
3010/3010 [==============================] - 242s 80ms/step - loss: 3.6583 - accuracy: 0.2604
Epoch 9/100
3010/3010 [==============================] - 200s 66ms/step - loss: 3.4160 - accuracy: 0.2917
Epoch 10/100
3010/3010 [======================

3010/3010 [==============================] - 139s 46ms/step - loss: 0.5353 - accuracy: 0.8634
Epoch 78/100
3010/3010 [==============================] - 139s 46ms/step - loss: 0.5333 - accuracy: 0.8636
Epoch 79/100
3010/3010 [==============================] - 152s 51ms/step - loss: 0.5295 - accuracy: 0.8655
Epoch 80/100
3010/3010 [==============================] - 151s 50ms/step - loss: 0.5338 - accuracy: 0.8628
Epoch 81/100
3010/3010 [==============================] - 145s 48ms/step - loss: 0.5316 - accuracy: 0.8624
Epoch 82/100
3010/3010 [==============================] - 145s 48ms/step - loss: 0.5263 - accuracy: 0.8647
Epoch 83/100
3010/3010 [==============================] - 145s 48ms/step - loss: 0.5289 - accuracy: 0.8630
Epoch 84/100
3010/3010 [==============================] - 146s 49ms/step - loss: 0.5242 - accuracy: 0.8651
Epoch 85/100
3010/3010 [==============================] - 153s 51ms/step - loss: 0.5244 - accuracy: 0.8642
Epoch 86/100
3010/3010 [==========================

In [10]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 27ms/step
I will leave if they have a little
